## Summing all the prime numbers below a given number

In [1]:
import time
import os

# Simple code

def if_prime(x):
    if x <= 1:
        return 0
    elif x <= 3:
        return x
    elif x % 2 == 0 or x % 3 == 0:
        return 0
    i = 5
    while i**2 <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return 0
        i += 6
    return x

def sum_primes(x):
    result = 0
    for i in range(x):
        result += if_prime(i)
    return result

number = int(os.environ.get("NUMBER", 2_500_000)) #valor a poner en .sb
suma = 0
N = 3 # number of loops

start = time.time()
for i in range(N):
    suma = sum(map(if_prime, list(range(number))))
stop = time.time()
tiempo = (stop - start) / N

print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)

tiempo = %timeit -r 2 -o -q sum_primes(number)
suma = sum_primes(number)
print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)


The prime sum below  2500000 is  219697708195  and the time taken is 4.8819262981414795
The prime sum below  2500000 is  219697708195  and the time taken is 4.62 s ± 16 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


In [5]:
# a) Paquete Numba
import os
import time
from numba import njit

@njit
def if_prime(x):
    if x <= 1:
        return 0
    elif x <= 3:
        return x
    elif x % 2 == 0 or x % 3 == 0:
        return 0
    i = 5
    while i * i <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return 0
        i += 6
    return x

def sum_primes(x):
    result = 0
    for i in range(x):
        result += if_prime(i)
    return result

number = int(os.environ.get("NUMBER", 2_500_000)) #valor a poner en .sb
N = 3

start = time.time()
for i in range(N):
    suma = sum_primes(number)
stop = time.time()

tiempo = (stop - start) / N

print("Numba: The prime sum below", number, "is", suma, "and the time taken is", tiempo)

tiempo = %timeit -r 2 -o -q sum_primes(number)
suma = sum_primes(number)
print("Numba: The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)

Numba: The prime sum below 2500000 is 219697708195 and the time taken is 0.6059597333272299
Numba: The prime sum below  2500000 is  219697708195  and the time taken is 583 ms ± 1.34 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


In [8]:
# b) Multiprocesing con pool
import os
import time
from numba import njit
from multiprocessing import Pool

@njit
def if_prime(x):
    if x <= 1:
        return 0
    elif x <= 3:
        return x
    elif x % 2 == 0 or x % 3 == 0:
        return 0
    i = 5
    while i * i <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return 0
        i += 6
    return x

def sum_primes_parallel(x, nproc):
    with Pool(processes=nproc) as p:
        result = p.map(if_prime, range(x))
    return sum(result)

if __name__ == "__main__":
    number = int(os.environ.get("NUMBER", 2_500_000)) #valor a poner en .sb
    N = 3
    nproc = int(os.environ.get("NPROC", 4))  # valor por defecto 4

    start = time.time()
    for i in range(N):
        suma = sum_primes_parallel(number, nproc)
    stop = time.time()

    tiempo = (stop - start) / N

    print("Multiprocessing: The prime sum below", number, "is", suma, "and the time taken is", tiempo)
    
    tiempo = %timeit -r 2 -o -q sum_primes(number)
    suma = sum_primes(number)
    print("Multiprocessing: The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)
    

Multiprocessing: The prime sum below 2500000 is 219697708195 and the time taken is 0.31586289405822754
Multiprocessing: The prime sum below  2500000 is  219697708195  and the time taken is 577 ms ± 1.62 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


In [9]:
# c) Numba con prange
import os
import time
from numba import njit, prange, set_num_threads, get_num_threads

# Núcleos
n_threads = int(os.environ.get("N_THREADS", 4))  # valor por defecto 4
set_num_threads(n_threads)

@njit
def if_prime(x):
    if x <= 1:
        return 0
    elif x <= 3:
        return x
    elif x % 2 == 0 or x % 3 == 0:
        return 0
    i = 5
    while i * i <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return 0
        i += 6
    return x

@njit(parallel=True)
def sum_primes(x):
    result = 0
    for i in prange(x):
        result += if_prime(i)
    return result

number = int(os.environ.get("NUMBER", 2_500_000)) #valor a poner en .sb
suma = 0
N = 3

start = time.time()
for i in range(N):
    suma = sum_primes(number)
stop = time.time()
tiempo = (stop - start) / N

print("Numba + prange: The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)

tiempo = %timeit -r 2 -o -q sum_primes(number)
suma = sum_primes(number)
print("Numba + prange: The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)

Numba + prange: The prime sum below  2500000 is  219697708195  and the time taken is 0.2161558469136556
Numba + prange: The prime sum below  2500000 is  219697708195  and the time taken is 70.6 ms ± 22.7 μs per loop (mean ± std. dev. of 2 runs, 10 loops each)


RESULTADOS PARA 10^6:

- 1 núcleo de ejecución:
The prime sum below 1000000 is 37550402023 and the time taken is 2.2635367711385093
The prime sum below 1000000 is 37550402023 and the time taken is 2.19 s ± 626 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)

Numba: The prime sum below 1000000 is 37550402023 and the time taken is 1.8533261617024739
Numba: The prime sum below 1000000 is 37550402023 and the time taken is 356 ms ± 764 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)

Multiprocessing: The prime sum below 1000000 is 37550402023 and the time taken is 0.527775764465332
Multiprocessing: The prime sum below 1000000 is 37550402023 and the time taken is 350 ms ± 552 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)

Numba + prange: The prime sum below 1000000 is 37550402023 and the time taken is 0.35942784945170086
Numba + prange: The prime sum below 1000000 is 37550402023 and the time taken is 115 ms ± 452 ns per loop (mean ± std. dev. of 2 runs, 10 loops each)

- 2 núcleos de ejecución:
The prime sum below 1000000 is 37550402023 and the time taken is 2.2557214895884194
The prime sum below 1000000 is 37550402023 and the time taken is 2.21 s ± 3.49 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)

Numba: The prime sum below 1000000 is 37550402023 and the time taken is 1.767690102259318
Numba: The prime sum below 1000000 is 37550402023 and the time taken is 357 ms ± 608 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)

Multiprocessing: The prime sum below 1000000 is 37550402023 and the time taken is 0.36142341295878094
Multiprocessing: The prime sum below 1000000 is 37550402023 and the time taken is 349 ms ± 347 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)

Numba + prange: The prime sum below 1000000 is 37550402023 and the time taken is 0.3162718613942464
Numba + prange: The prime sum below 1000000 is 37550402023 and the time taken is 70.6 ms ± 86 μs per loop (mean ± std. dev. of 2 runs, 10 loops each)

- 4 núcleos de ejecución:
The prime sum below 1000000 is 37550402023 and the time taken is 2.2672006289164224
The prime sum below 1000000 is 37550402023 and the time taken is 2.28 s ± 1.14 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)

Numba: The prime sum below 1000000 is 37550402023 and the time taken is 1.8454070091247559
Numba: The prime sum below 1000000 is 37550402023 and the time taken is 358 ms ± 1.9 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)

Multiprocessing: The prime sum below 1000000 is 37550402023 and the time taken is 0.2799201011657715
Multiprocessing: The prime sum below 1000000 is 37550402023 and the time taken is 350 ms ± 238 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)

Numba + prange: The prime sum below 1000000 is 37550402023 and the time taken is 0.28299880027770996
Numba + prange: The prime sum below 1000000 is 37550402023 and the time taken is 37.6 ms ± 2.09 μs per loop (mean ± std. dev. of 2 runs, 10 loops each)

- 8 núcleos de ejecución:
The prime sum below 1000000 is 37550402023 and the time taken is 2.262571414311727
The prime sum below 1000000 is 37550402023 and the time taken is 2.19 s ± 187 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)

Numba: The prime sum below 1000000 is 37550402023 and the time taken is 1.8519644737243652
Numba: The prime sum below 1000000 is 37550402023 and the time taken is 357 ms ± 580 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)

Multiprocessing: The prime sum below 1000000 is 37550402023 and the time taken is 0.2653204600016276
Multiprocessing: The prime sum below 1000000 is 37550402023 and the time taken is 356 ms ± 965 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)

Numba + prange: The prime sum below 1000000 is 37550402023 and the time taken is 0.2640852133433024
Numba + prange: The prime sum below 1000000 is 37550402023 and the time taken is 19.7 ms ± 33 μs per loop (mean ± std. dev. of 2 runs, 100 loops each)


En base a estos resultados podemos concluir que el código original es lento y no mejora al aumentar los núcleos porque no está paralelizado. Numba secuencial acelera el cálculo pero tampoco escala con múltiples núcleos al ejecutarse en un solo hilo por defecto. Multiprocessing con Pool reduce el tiempo pero el overhead de creación de procesos y comunicación limita la mejora al aumentar núcleos para este tamaño de problema. La combinación Numba + prange es la más eficiente, optimizando el tiempo de ejecución hasta 0.264 ms en 8 núcleos, aprovechando efectivamente la paralelización y escalando casi linealmente.

RESULTADOS PARA 10^7:

- 1 núcleo de ejecución:
The prime sum below 10000000 is 3203324994356 and the time taken is 59.14328614870707
The prime sum below 10000000 is 3203324994356 and the time taken is 57.3 s ± 16.6 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)

Numba: The prime sum below 10000000 is 3203324994356 and the time taken is 6.541407426198323
Numba: The prime sum below 10000000 is 3203324994356 and the time taken is 5.05 s ± 6.01 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)

Multiprocessing: The prime sum below 10000000 is 3203324994356 and the time taken is 5.5963443120320635
Multiprocessing: The prime sum below 10000000 is 3203324994356 and the time taken is 5.01 s ± 39.5 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)

Numba + prange: The prime sum below 10000000 is 3203324994356 and the time taken is 3.113591273625692
Numba + prange: The prime sum below 10000000 is 3203324994356 and the time taken is 2.87 s ± 7.13 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)

- 2 núcleos de ejecución:
The prime sum below 10000000 is 3203324994356 and the time taken is 59.71988224983215
The prime sum below 10000000 is 3203324994356 and the time taken is 57.7 s ± 5.65 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)

Numba: The prime sum below 10000000 is 3203324994356 and the time taken is 6.51400105158488
Numba: The prime sum below 10000000 is 3203324994356 and the time taken is 5.12 s ± 4.73 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)

Multiprocessing: The prime sum below 10000000 is 3203324994356 and the time taken is 3.1234613259633384
Multiprocessing: The prime sum below 10000000 is 3203324994356 and the time taken is 5.02 s ± 1.52 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)

Numba + prange: The prime sum below 10000000 is 3203324994356 and the time taken is 2.034602959950765
Numba + prange: The prime sum below 10000000 is 3203324994356 and the time taken is 1.79 s ± 11 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)

- 4 núcleos de ejecución:
The prime sum below 10000000 is 3203324994356 and the time taken is 61.30626837412516
The prime sum below 10000000 is 3203324994356 and the time taken is 58.6 s ± 79.8 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)

Numba: The prime sum below 10000000 is 3203324994356 and the time taken is 6.560088237126668
Numba: The prime sum below 10000000 is 3203324994356 and the time taken is 5.07 s ± 15.3 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)

Multiprocessing: The prime sum below 10000000 is 3203324994356 and the time taken is 1.7998578548431396
Multiprocessing: The prime sum below 10000000 is  3203324994356 and the time taken is 5.01 s ± 817 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)

Numba + prange: The prime sum below 10000000 is 3203324994356 and the time taken is 1.2017743587493896
Numba + prange: The prime sum below 10000000 is 3203324994356 and the time taken is 957 ms ± 165 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)
  
- 8 núcleos de ejecución:
The prime sum below  10000000 is  3203324994356  and the time taken is 60.27457284927368
The prime sum below  10000000 is  3203324994356  and the time taken is 58.5 s ± 472 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)

Numba: The prime sum below 10000000 is 3203324994356 and the time taken is 6.577128330866496
Numba: The prime sum below  10000000 is  3203324994356  and the time taken is 5.05 s ± 3.71 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)

Multiprocessing: The prime sum below 10000000 is 3203324994356 and the time taken is 1.2147163550059001
Multiprocessing: The prime sum below  10000000 is  3203324994356  and the time taken is 5.02 s ± 12.4 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)

Numba + prange: The prime sum below  10000000 is  3203324994356  and the time taken is 0.7440750598907471
Numba + prange: The prime sum below  10000000 is  3203324994356  and the time taken is 519 ms ± 26.6 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)

En este caso, el código original tarda sobre 57–58 s y no mejora al aumentar núcleos, reflejando que no está paralelizado. Numba con el decorador @njit reduce el tiempo pero sigue ejecutándose en un solo hilo, por lo que la variación de núcleos no tiene impacto. Multiprocessing con Pool empieza a mostrar mejoras al aumentar núcleos, especialmente en 4 y 8 núcleos, pero los tiempos aún se ven afectados por el overhead de procesos, por lo que la ganancia no es totalmente lineal. Numba + prange combina compilación JIT con paralelización efectiva, reduciendo el tiempo de ejecución hasta 0.744 en 8 núcleos, mostrando la mejor escalabilidad y eficiencia.